## EZKL HUB Upload Artifacts Jupyter Notebook Demo 

After you have setup your a circuit locally by running `generate_hub_artifacts.ipynb`, you can upload it to the EZKL hub. This is useful in cases where you want to debug and finetune your circuit params locally before uploading it to the hub for delegated proving. 
<!-- 
Here we demonstrate the use of the EZKL hub in a Jupyter notebook whereby all components of the circuit are public or pre-committed to. This is the simplest case of using EZKL (proof of computation). -->

This will be accomplished in 7 steps (steps 1-6 are accomplish by `generate_hub_artifacts.ipynb`). 

1. Train the model. 
2. Export the model to onnnx
3. Generate then calibrate the circuit settings.
4. Compile the circuit.
5. Download SRS
6. Setup circuit keys
7. Upload the compiled model, settings and proving keys to the hub. 


In [ ]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


# here we create and (potentially train a model)

import os

In [ ]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('key.pk')
vk_path = os.path.join('key.vk')
settings_path = os.path.join('settings.json')
srs_path = os.path.join('kzg.srs')
witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [ ]:
import time
import ezkl

# Now that we have all the circuit setup artifacts, we can upload them to the hub

test_hub_name = "samtvlabs" #we've set this up for you, but you can create your own hub name and use that instead

organization = ezkl.get_hub_credentials(test_hub_name)['organizations'][0]

print("organization: " + str(organization))

# timestamped name
name = "model_" + str(int(time.time()))

deployed_model = ezkl.upload_hub_artifact(compiled_model_path, settings_path, pk_path, name, organization['id'])

print("deployed model: " + str(deployed_model))

In [ ]:
# Loop every 5 seconds until status is not pending
status = "PENDING"
while status == "PENDING":
    time.sleep(5)
    get_model = ezkl.get_hub_artifact(deployed_model['id'])
    status = get_model['status']

In [ ]:

proof_id = ezkl.prove_hub(deployed_model['id'], data_path)
print("proof id: " + str(proof_id))

In [ ]:
# Loop every 5 seconds until status is not pending
status = "PENDING"
while status == "PENDING":
    time.sleep(5)
    get_proof = ezkl.get_hub_proof(proof_id['id'])
    status = get_proof['status']

In [ ]:
proof = ezkl.get_hub_proof(proof_id['id'])

print("proof: " + str(proof))